In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Randomly seeding
tf.random.set_seed(6950)
# Constants
SAMPLING_RATE = 500  # Hz
ecg_folder = "../../../Datasets/12-lead electrocardiogram database/ECGData"
diagnostics_file = "../../../Datasets/12-lead electrocardiogram database/Diagnostics.xlsx"
# Label mapping
# Basically to reduce 11 label to 4 for better performance on chapmanecg dataset
rhythm_mapping = {
    'AFIB': 'AFIB',
    'AF': 'AFIB',
    'SVT': 'GSVT',
    'AT': 'GSVT',
    'SAAWR': 'GSVT',
    'ST': 'GSVT',
    'AVNRT': 'GSVT',
    'AVRT': 'GSVT',
    'SB': 'SB',
    'SR': 'SR',
    'SA': 'SR'
}

2024-12-24 10:28:03.516720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-24 10:28:03.527522: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-24 10:28:03.530940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-24 10:28:03.539934: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-24 10:28:04.087117: W tensorflow/compiler/tf2

In [2]:
def hamilton_tompkins_qrs_detector(ecg_signal, sampling_rate=500):
    """
    Implementation of Hamilton-Tompkins QRS detection algorithm

    Args:
        ecg_signal: Raw ECG signal
        sampling_rate: Sampling frequency in Hz

    Returns:
        qrs_peaks: Array of QRS peak locations
        filtered_ecg: Filtered ECG signal for further analysis
    """
    # Bandpass filter (5-15 Hz)
    nyquist_freq = sampling_rate / 2
    low = 5 / nyquist_freq
    high = 15 / nyquist_freq
    b, a = signal.butter(3, [low, high], btype='band')
    filtered_ecg = signal.filtfilt(b, a, ecg_signal)

    # Derivative
    derivative = np.diff(filtered_ecg)

    # Squaring
    squared = derivative ** 2

    # Moving window integration
    window_size = int(0.150 * sampling_rate)  # 150 ms window
    window = np.ones(window_size) / window_size
    integrated = np.convolve(squared, window, mode='same')

    # Adaptive thresholding
    peak_threshold = 0.3 * np.max(integrated)

    # Find peaks
    qrs_peaks, _ = signal.find_peaks(integrated, height=peak_threshold, distance=sampling_rate//2)

    return qrs_peaks, filtered_ecg

def detect_p_t_waves(filtered_ecg, qrs_peaks, sampling_rate=500):
    """
    Detect P and T waves in the ECG signal

    Args:
        filtered_ecg: Filtered ECG signal
        qrs_peaks: QRS peak locations
        sampling_rate: Sampling frequency in Hz

    Returns:
        p_peaks: Array of P wave peak locations
        t_peaks: Array of T wave peak locations
    """
    p_peaks = []
    t_peaks = []

    for qrs_peak in qrs_peaks:
        # Search window for P wave (200ms before QRS)
        p_search_start = max(0, qrs_peak - int(0.2 * sampling_rate))
        p_search_end = qrs_peak
        p_window = filtered_ecg[p_search_start:p_search_end]
        if len(p_window) > 0:
            p_peak = p_search_start + np.argmax(p_window)
            p_peaks.append(p_peak)

        # Search window for T wave (400ms after QRS)
        t_search_start = qrs_peak
        t_search_end = min(len(filtered_ecg), qrs_peak + int(0.4 * sampling_rate))
        t_window = filtered_ecg[t_search_start:t_search_end]
        if len(t_window) > 0:
            t_peak = t_search_start + np.argmax(t_window)
            t_peaks.append(t_peak)

    return np.array(p_peaks), np.array(t_peaks)

def calculate_axis(ecg_data):
    """
    Calculate R and T axis (simplified approximation)
    """
    # This is a simplified calculation - in practice, you'd need multiple leads
    r_axis = np.mean(np.arctan2(ecg_data[:, 1], ecg_data[:, 0])) * 180 / np.pi
    t_axis = r_axis + np.random.normal(0, 15)  # Simplified approximation
    return r_axis, t_axis

def extract_features(ecg_data, sampling_rate=500):
    """
    Extract all features from ECG signal

    Args:
        ecg_data: Raw ECG data (lead II is in column 1, after header row)
        sampling_rate: Sampling frequency in Hz

    Returns:
        features: Dictionary containing all extracted features
    """
    # Get lead II data
    lead_II = ecg_data[:, 1]

    # Get QRS peaks and filtered signal
    qrs_peaks, filtered_ecg = hamilton_tompkins_qrs_detector(lead_II, sampling_rate)

    # Get P and T waves
    p_peaks, t_peaks = detect_p_t_waves(filtered_ecg, qrs_peaks, sampling_rate)

    features = {}

    # Ventricular rate (bpm)
    if len(qrs_peaks) > 1:
        rr_intervals = np.diff(qrs_peaks) / sampling_rate
        features['VentricularRate'] = 60 / np.mean(rr_intervals)
    else:
        features['VentricularRate'] = 0

    # Atrial rate (bpm)
    if len(p_peaks) > 1:
        pp_intervals = np.diff(p_peaks) / sampling_rate
        features['AtrialRate'] = 60 / np.mean(pp_intervals)
    else:
        features['AtrialRate'] = 0

    # QRS duration
    qrs_duration = []
    for peak in qrs_peaks:
        # Find QRS onset and offset
        start = max(0, peak - int(0.1 * sampling_rate))
        end = min(len(filtered_ecg), peak + int(0.1 * sampling_rate))
        qrs_window = filtered_ecg[start:end]
        if len(qrs_window) > 0:
            # Use threshold crossing to estimate duration
            threshold = 0.1 * np.max(qrs_window)
            above_threshold = qrs_window > threshold
            if np.any(above_threshold):
                first_cross = np.where(above_threshold)[0][0]
                last_cross = np.where(above_threshold)[0][-1]
                qrs_duration.append((last_cross - first_cross) / sampling_rate * 1000)  # in ms

    features['QRSDuration'] = np.mean(qrs_duration) if qrs_duration else 0

    # QT interval
    qt_intervals = []
    for qrs, t in zip(qrs_peaks[:len(t_peaks)], t_peaks):
        qt_intervals.append((t - qrs) / sampling_rate * 1000)  # in ms

    features['QTInterval'] = np.mean(qt_intervals) if qt_intervals else 0

    # QTc using Bazett's formula
    if features['VentricularRate'] > 0:
        features['QTCorrected'] = features['QTInterval'] * np.sqrt(60 / features['VentricularRate'])
    else:
        features['QTCorrected'] = 0

    # Calculate R and T axis
    features['RAxis'], features['TAxis'] = calculate_axis(ecg_data)

    # QRS Count
    features['QRSCount'] = len(qrs_peaks)

    # Q wave timing features
    if len(qrs_peaks) > 0:
        features['QOnset'] = qrs_peaks[0] / sampling_rate * 1000  # in ms
        features['QOffset'] = (qrs_peaks[0] + features['QRSDuration'] * sampling_rate / 1000) / sampling_rate * 1000
    else:
        features['QOnset'] = 0
        features['QOffset'] = 0

    # T wave offset
    if len(t_peaks) > 0:
        features['TOffset'] = t_peaks[-1] / sampling_rate * 1000  # in ms
    else:
        features['TOffset'] = 0

    return features

def load_and_preprocess_data(ecg_folder, diagnostics_file, rhythm_mapping):
    """
    Load and preprocess the ECG data and diagnostics
    """
    # Read diagnostics file
    diagnostics = pd.read_excel(diagnostics_file)

    # Filter relevant columns and map rhythms
    diagnostics['Rhythm'] = diagnostics['Rhythm'].map(rhythm_mapping)

    # Initialize lists for features and labels
    all_features = []
    labels = []

    # Process each ECG file
    for idx, row in diagnostics.iterrows():
        file_path = os.path.join(ecg_folder, row['FileName'] + '.csv')
        if os.path.exists(file_path):
            # Load ECG data, skip header row
            ecg_data = np.loadtxt(file_path, delimiter=',', skiprows=1)

            # Extract features
            features = extract_features(ecg_data)

            # Create feature vector in the same order as diagnostics
            feature_vector = [
                features['VentricularRate'],
                features['AtrialRate'],
                features['QRSDuration'],
                features['QTInterval'],
                features['QTCorrected'],
                features['RAxis'],
                features['TAxis'],
                features['QRSCount'],
                features['QOnset'],
                features['QOffset'],
                features['TOffset']
            ]

            all_features.append(feature_vector)
            labels.append(row['Rhythm'])

    return np.array(all_features), np.array(labels)

def create_model(input_shape, num_classes):
    """
    Create a simple MLP model
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_shape,)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Load and preprocess data
X, y = load_and_preprocess_data(ecg_folder, diagnostics_file, rhythm_mapping)

# Convert labels to numerical
label_to_id = {label: idx for idx, label in enumerate(np.unique(y))}
y_encoded = np.array([label_to_id[label] for label in y])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train model
model = create_model(X_train.shape[1], len(label_to_id))

history = model.fit(
    X_train_scaled, y_train,
    epochs=200,
    batch_size=128,
    validation_split=0.2,
    # callbacks=[
    #     tf.keras.callbacks.EarlyStopping(
    #         monitor='val_loss',
    #         patience=5,
    #         restore_best_weights=True
    #     )
    # ]
)

# Evaluate model
y_pred = np.argmax(model.predict(X_test_scaled), axis=1)

# Convert numerical labels back to original classes
id_to_label = {v: k for k, v in label_to_id.items()}
y_test_original = [id_to_label[y] for y in y_test]
y_pred_original = [id_to_label[y] for y in y_pred]

# Print classification report
print(classification_report(y_test_original, y_pred_original))


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1735014527.143128  463645 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735014527.175317  463645 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1735014527.

Epoch 1/200


I0000 00:00:1735014527.986538  464186 service.cc:146] XLA service 0x79e45c00c2f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735014527.986577  464186 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-12-24 10:28:48.005558: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-24 10:28:48.103009: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 1/54 ━━━━━━━━━━━━━━━━━━━━ 1:39 2s/step - accuracy: 0.2188 - loss: 1.4721

I0000 00:00:1735014529.303731  464186 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 82ms/step - accuracy: 0.4068 - loss: 1.2843 - val_accuracy: 0.6238 - val_loss: 0.8784
Epoch 2/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6269 - loss: 0.8754 - val_accuracy: 0.6942 - val_loss: 0.7604
Epoch 3/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6686 - loss: 0.7947 - val_accuracy: 0.7160 - val_loss: 0.7191
Epoch 4/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7012 - loss: 0.7428 - val_accuracy: 0.7342 - val_loss: 0.6829
Epoch 5/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7035 - loss: 0.7109 - val_accuracy: 0.7330 - val_loss: 0.6740
Epoch 6/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7156 - loss: 0.6965 - val_accuracy: 0.7400 - val_loss: 0.6550
Epoch 7/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7153 - loss: 0.6824 - val_accuracy: 0.7353 - val_loss: 0.6444
Epoch 8/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7263 - loss: 0.6677 - val_accuracy: 0.7347 - val_loss: 0.